# 第6章: 機械学習

> 本章では，Fabio Gasparetti氏が公開しているNews Aggregator Data Setを用い，  
> ニュース記事の見出しを「ビジネス」「科学技術」「エンターテイメント」「健康」の  
> カテゴリに分類するタスク（カテゴリ分類）に取り組む．

In [ ]:
import os
import re
import sys
import random
from pprint import pprint
from tqdm import tqdm


import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, confusion_matrix, precision_score, 
                             recall_score, f1_score, precision_recall_fscore_support, 
                             classification_report)

%matplotlib inline

In [ ]:
DATADIR = "data"          # データを保存するおおもとのディレクトリ
CURRENTDIR = "/workspace" # notebookを置いているディレクトリ

# シード値の固定
SEED = 19950908
random.seed(SEED)

## 50. データの入手・整形

> News Aggregator Data Setをダウンロードし、以下の要領で学習データ（train.txt），検証データ（valid.txt），評価データ（test.txt）を作成せよ．
> 1. ダウンロードしたzipファイルを解凍し，readme.txtの説明を読む．>
> 2. 情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．
> 3. 抽出された事例をランダムに並び替える．
> 4. 抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．ファイルには，１行に１事例を書き出すこととし，カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．
> 
> 学習データと評価データを作成したら，各カテゴリの事例数を確認せよ．

In [ ]:
# 6章で利用するデータを保管するディレクトリの作成
CHAPDIR = os.path.join(DATADIR, "chapter6")
try:
    os.mkdir(CHAPDIR)
except:
    print("作成済み等の理由でディレクトリが作成されませんでした")

In [ ]:
# ディレクトリの移動
%cd $CURRENTDIR/$CHAPDIR

# データのダウンロード
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/00359/NewsAggregatorDataset.zip
!unzip NewsAggregatorDataset.zip

In [ ]:
# ディレクトリの移動
%cd $CURRENTDIR/$CHAPDIR
# 行数の確認
!wc -l ./newsCorpora.csv
# 先頭10行の確認
!head -10 ./newsCorpora.csv


---
### データの抽出
newsCorpora.csvのカラム情報（すべてタブ区切り。なんでcsvやねん）

- ID
- TITLE
- URL
- PUBLISHER
- CATEGORY
- STORY
- HOSTNAME
- TIMESTAMP


以下のように指定されているので、抽出しながらデータを持つ
> 情報源（publisher）が”Reuters”, “Huffington Post”, “Businessweek”, “Contactmusic.com”, “Daily Mail”の事例（記事）のみを抽出する．

In [ ]:
# カレントディレクトリをもどす
%cd $CURRENTDIR

# データの読み込み
news_fpath = os.path.join(CHAPDIR, "newsCorpora.csv")
# 抽出条件
pub_filter = ["Reuters", "Huffington Post", "Businessweek", "Contactmusic.com", "Daily Mail"]
# 利用する全データ
all_data = []

with open(news_fpath, "r", encoding="utf8")as fr:
    for line in fr:
        # タブ区切りをリストで取得
        record = line.rstrip("\n").split("\t")
        if record[3] in pub_filter:
            #条件に当てはまるなら追加
            all_data.append(record)

print("使用するデータ数：", len(all_data))
pprint(all_data[:3])

### 学習、検証、訓練データに分割

以下の処理を行う

> 抽出された事例をランダムに並び替える．
> 
> 抽出された事例の80%を学習データ，残りの10%ずつを検証データと評価データに分割し，  
> それぞれtrain.txt，valid.txt，test.txtというファイル名で保存する．  
> ファイルには，１行に１事例を書き出すこととし，  
> カテゴリ名と記事見出しのタブ区切り形式とせよ（このファイルは後に問題70で再利用する）．


In [ ]:
# 学習データやらの分割
# 8:1:1で分割
train_data, val_test_data = train_test_split(all_data, test_size=0.2, shuffle=True, 
                                             random_state=SEED, stratify=[r[4] for r in all_data])
val_data, test_data = train_test_split(val_test_data, test_size=0.5, shuffle=True, 
                                       random_state=SEED, stratify=[r[4] for r in val_test_data])
# 確認
print(len(train_data), len(val_data), len(test_data))

In [ ]:
# データの出力
fname_lst = ["train.txt", "valid.txt", "test.txt"]
split_data_lst = [train_data, val_data, test_data]

for fname, data in zip(fname_lst, split_data_lst):
    # 出力先のパス生成
    output_fpath = os.path.join(CHAPDIR, fname)
    with open(output_fpath, "w", encoding="utf8")as fw:
        for record in data:
            # カテゴリとタイトルをタブ区切り
            output_line = "\t".join([record[4], record[1]])
            fw.write(f"{output_line}\n")

In [ ]:
# 出力したデータの確認
all_data_dct = {}

for fname in fname_lst:
    # 入力ファイル
    input_fpath  =os.path.join(CHAPDIR, fname)
    with open(input_fpath, "r", encoding="utf8")as fr:
        # タブ区切りをリストで取得
        data = [line.rstrip("\n").split("\t") for line in fr]
        all_data_dct[fname] = data

# 各カテゴリの事例数を確認する
for fname, data in all_data_dct.items():
    print("【{}】".format(fname))
    pprint(collections.Counter([record[0] for record in data]))

## 51. 特徴量抽出

> 学習データ，検証データ，評価データから特徴量を抽出し，それぞれtrain.feature.txt，valid.feature.txt，test.feature.txtというファイル名で保存せよ． なお，カテゴリ分類に有用そうな特徴量は各自で自由に設計せよ．記事の見出しを単語列に変換したものが最低限のベースラインとなるであろう．

とりあえず英語のストップワードとTF-IDFでやる。

前処理に関しては以下の内容に取り組む
- 小文字化
- 「's」「...」「-」の削除
- 数字の0化

In [ ]:
# 前処理用の正規表現
re_del = re.compile("('s|-|\.\.\.)")
re_num = re.compile(r"[０-９0-9]([０-９．,0-9.,]?[０-９0-9])*")

# 前処理関数を作成
def preprocessing(text):
    text = text.lower()          # 小文字化
    text = re_del.sub(" ", text) # 特定文字の削除 
    text = re_num.sub("0", text) # 数字の0化
    return text

In [ ]:
# 全データに前処理の適応

# 訓練データ
train_title_lst = []
y_train = []
for record in all_data_dct["train.txt"]:
    train_title_lst.append(preprocessing(record[1]))
    y_train.append(record[0])

# 検証データ
val_title_lst = []
y_val = []
for record in all_data_dct["valid.txt"]:
    val_title_lst.append(preprocessing(record[1]))
    y_val.append(record[0])

# 試験データ
test_title_lst = []
y_test = []
for record in all_data_dct["test.txt"]:
    test_title_lst.append(preprocessing(record[1]))
    y_test.append(record[0])            

In [ ]:
# ベクタライザーのインスタンス化
vectorizer = TfidfVectorizer(min_df=5, ngram_range=(1, 2), stop_words="english")

# 訓練+検証データでTF-IDFベクトルの作成
X_train_val = vectorizer.fit_transform(train_title_lst+val_title_lst)
# 試験データに適応
X_test = vectorizer.transform(test_title_lst)

# 訓練データと検証データの再分割
X_train = X_train_val[:len(train_title_lst)]
X_val = X_train_val[len(train_title_lst):]

In [ ]:
# ファイルへの出力
fname_lst = ["train.feature.txt", "valid.feature.txt", "test.feature.txt"]
split_X_lst = [X_train, X_val, X_test]

for fname, X_data in zip(fname_lst, split_X_lst):
    # 出力するファイルパス
    output_fpath = os.path.join(CHAPDIR, fname)
    # sparseデータをCSVにするのにpandasが実装上手っ取り早いので
    df_temp = pd.DataFrame(X_data.toarray(), columns=vectorizer.get_feature_names())
    # 出力
    df_temp.to_csv(output_fpath, sep='\t', index=False)        

## 52. 学習

> 51で構築した学習データを用いて，ロジスティック回帰モデルを学習せよ．

In [ ]:
lg_model = LogisticRegression(random_state=SEED, max_iter=10000)
lg_model.fit(X_train, y_train)

## 53. 予測

> 52で学習したロジスティック回帰モデルを用い，与えられた記事見出しからカテゴリとその予測確率を計算するプログラムを実装せよ．

In [ ]:
# 試し
lg_model.predict_proba(X_test[:10])

In [ ]:
def get_predict_score(lg_model, X):
    return (np.max(lg_model.predict_proba(X), axis=1), lg_model.predict(X))

get_predict_score(lg_model, X_train[:5])

## 54. 正解率の計測

> 52で学習したロジスティック回帰モデルの正解率を，学習データおよび評価データ上で計測せよ

In [ ]:
# 予測値の取得
pred_train = get_predict_score(lg_model, X_train)
pred_test = get_predict_score(lg_model, X_test)

acc_train = accuracy_score(y_train, pred_train[1])
acc_test = accuracy_score(y_test, pred_test[1])

print(f"accuracy（train）：{acc_train:.3f}")
print(f'accuracy（test）：{acc_test:.3f}')

## 55. 混同行列の作成

> 52で学習したロジスティック回帰モデルの混同行列（confusion matrix）を，学習データおよび評価データ上で作成せよ．

In [ ]:
# 学習データ
train_conf_mat = confusion_matrix(y_train, pred_train[1])
sns.heatmap(train_conf_mat, annot=True, cmap="Blues")
plt.show()

# 評価データ
test_conf_mat = confusion_matrix(y_test, pred_test[1])
sns.heatmap(test_conf_mat, annot=True, cmap="Blues")
plt.show()

## 56. 適合率，再現率，F1スコアの計測

> 52で学習したロジスティック回帰モデルの適合率，再現率，F1スコアを，評価データ上で計測せよ．  
> カテゴリごとに適合率，再現率，F1スコアを求め，カテゴリごとの性能をマイクロ平均（micro-average）とマクロ平均（macro-average）で統合せよ．

In [ ]:
# micro avgとaccuracyが基本一致する
print(classification_report(y_test, pred_test[1]))

## 57. 特徴量の重みの確認

> 52で学習したロジスティック回帰モデルの中で，重みの高い特徴量トップ10と，重みの低い特徴量トップ10を確認せよ．

In [ ]:
# 特徴次元の取得
vocabs = vectorizer.get_feature_names()
# 重要度の取得
for c, coefs in zip(lg_model.classes_, lg_model.coef_):
    print("----- カテゴリ：{} -----".format(c))
    # 重要度が高い10件
    top10_idxs = np.argsort(coefs)[::-1][:10]
    print("重要度上位：", ", ".join([vocabs[idx] for idx in top10_idxs]))
    # 重要度が低い10件
    bottom10_idxs = np.argsort(coefs)[:10]
    print("重要度下位：", ", ".join([vocabs[idx] for idx in bottom10_idxs]))

## 58. 正則化パラメータの変更

ここはQiitaの回答丸パクリ

> ロジスティック回帰モデルを学習するとき，正則化パラメータを調整することで，  
> 学習時の過学習（overfitting）の度合いを制御できる．  
> 異なる正則化パラメータでロジスティック回帰モデルを学習し，学習データ，検証データ，および評価データ上の正解率を求めよ．  
> 実験の結果は，正則化パラメータを横軸，正解率を縦軸としたグラフにまとめよ．

In [ ]:
# 出力用リスト
results = []
for C in tqdm(np.logspace(-5, 4, 10, base=10)):
    # 学習
    lg_model = LogisticRegression(random_state=SEED, max_iter=10000, C=C)
    lg_model.fit(X_train, y_train)
    
    # 予測
    pred_train = get_predict_score(lg_model, X_train)
    pred_val = get_predict_score(lg_model, X_val)
    pred_test = get_predict_score(lg_model, X_test)
    
    # accuracyを求める
    acc_train = accuracy_score(y_train, pred_train[1])
    acc_val = accuracy_score(y_val, pred_val[1])
    acc_test = accuracy_score(y_test, pred_test[1])
    
    # 結果の格納
    results.append([C, acc_train, acc_val, acc_test])

In [ ]:
results

In [ ]:
# 可視化
results = np.array(results).T
plt.plot(results[0], results[1], label='train')
plt.plot(results[0], results[2], label='valid')
plt.plot(results[0], results[3], label='test')
plt.ylim(0, 1.1)
plt.ylabel('Accuracy')
plt.xscale ('log')
plt.xlabel('C')
plt.legend()
plt.show()

## 59. ハイパーパラメータの探索

> 学習アルゴリズムや学習パラメータを変えながら，カテゴリ分類モデルを学習せよ．検証データ上の正解率が最も高くなる学習アルゴリズム・パラメータを求めよ．また，その学習アルゴリズム・パラメータを用いたときの評価データ上の正解率を求めよ．
